In [1]:
import sys
sys.path.append("../")

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

from torch.nn.utils.rnn import pack_padded_sequence
import torchvision as tv
import torch.utils.data as td
from torch.nn import functional as F
from torch import nn
import torch

import models.nntools_modified as nt
import models.framework as fw
import models.framework_varying as fwv
import models.framework_single_varying as fwsv
import utils.config as cfg
from utils.datareader_toy import toyScenesdata

from utils.visualization import *

from PIL import Image
import cv2
# import imageio

cuda
cuda
cuda


In [8]:
exp_name = "res_gpu_vary_epochs_150_downsample_pt_2_5_data2/"

net = fwv.MultiAgentNetwork(cfg.FCN_OUT, cfg.NUM_INTENTS, cfg.SCENE_OUT,
                                cfg.INTENT_IN, cfg.INTENT_OUT, cfg.SCORE_IN)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

net = net.to(device)
test_set = toyScenesdata(set_name="test")
# output_dir = "data/res_gpu_original_downsample_epoch_75_pt_5/" # output_dir = cfg.EXP_NAME
output_dir = cfg.OUTPUT_PATH + exp_name

optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)

Read in 1000 
700
200
100
Read in 1000 
700
200
100


In [9]:
def load_state_dict(checkpoint):
    """Loads the experiment from the input checkpoint."""
    net.load_state_dict(checkpoint['Net'])
#     optimizer.load_state_dict(checkpoint['Optimizer'])
    history = checkpoint['History']

    # The following loops are used to fix a bug that was
    # discussed here: https://github.com/pytorch/pytorch/issues/2830
    # (it is supposed to be fixed in recent PyTorch version)
    for state in optimizer.state.values():
        for k, v in state.items():
            if isinstance(v, torch.Tensor):
                state[k] = v.to(net.device)

def load():
    """Loads the experiment from the last checkpoint saved on disk."""
    checkpoint = torch.load(checkpoint_path,
                            map_location=net.device)
    load_state_dict(checkpoint)
#     del checkpoint

In [10]:
# Initialize history
history = []

# Define checkpoint paths
# if output_dir is None:
#     output_dir = 'experiment_{}'.format(time.time())
# os.makedirs(output_dir, exist_ok=True)
checkpoint_path = os.path.join(output_dir, "checkpoint.pth.tar")
config_path = os.path.join(output_dir, "config.txt")

# Load checkpoint and check compatibility
# if os.path.isfile(config_path):
#     with open(config_path, 'r') as f:
#         if f.read()[:-1] != repr(self):
#             raise ValueError(
#                 "Cannot create this experiment: "
#                 "I found a checkpoint conflicting with the current setting.")
load()

FileNotFoundError: [Errno 2] No such file or directory: '../outputs/res_gpu_vary_epochs_150_downsample_pt_2_5_data2/checkpoint.pth.tar'

In [ ]:
# ls - the entire groundtruth trajectory of the agent
# vs - the entire groundtruth velocities of the agent

# ls, vs -> required for plotting the actual trajectories
# vs additionally required for the initial velocity value
# s_points -> refers to the actual data that is used in init.npy file.
# these are the starting conditions set initially for the image

def getCollided(index, num_agents=2):
    data_path = cfg.DATA_PATH + f"{num_agents}/" + str(index) + "/"
    base_path = cfg.ACTUAL_PATH + f"/{num_agents}/" + str(index) + "/"
    
    s_points = np.load(base_path + "init.npy", allow_pickle=True)
    vs = np.load(base_path + "vs.npy")
    ls = np.load(base_path + "ls.npy")
    # im = imageio.imread("../data/toydataset/2/2716/scene.png")
#     im = np.array(cv2.imread(base_path + "scene.png"))
    
    # sample_img = imageio.imread("../data/toydataset_resampled/2/2716/scene.png")
    sample_img = np.array(cv2.imread(data_path + "scene.png"))
    sample_ls = np.load(data_path + "ls.npy")
    sample_gt = np.array(list(np.array(np.load(data_path + "init.npy", allow_pickle=True))))
#     print(sample_img.shape, sample_ls.shape, sample_gt.shape)

    sample_img = torch.from_numpy(sample_img)
    sample_img = sample_img.view(1, *sample_img.size())
#     print(sample_img.shape)
    sample_ls = torch.from_numpy(sample_ls)
    sample_ls = sample_ls.view(1, *sample_ls.size())
#     print(sample_ls.shape)
    sample_gt = torch.from_numpy(sample_gt)
    sample_gt = sample_gt.view(1, *sample_gt.size())
#     print(sample_gt.shape)
    sample_img = sample_img.to(net.device)
    sample_ls = sample_ls.to(net.device)
    sample_gt = sample_gt.to(net.device)
    
    #Get Output
    y, _ = net(sample_img, sample_ls, sample_gt)
    
    #Run output through a softmax layer
    s = nn.Softmax(dim=1)
    sout = s(y)
    
    n_modes = y.shape[1]
    nm = n_modes
    
    overall_ind = torch.argmax(sout, dim=1)
    ints = list()
    oin = overall_ind[0]
    for k in range(n_agents):
        oin = oin//4**k
        ints.append((oin%4).item())
        
    _, _, c = visualize(np.array(ints), s_points, ls, vs)
    return c

In [ ]:
num_agents = [2,3,4]
totalElements = 0
totalCollisions = 0


for agent in num_agents:
    subsetCollisions = 0
    
    test_dataset = toyScenesdata(set_name="test", agents=agent)
    totalElements += len(test_dataset)
    
    scenes = test_dataset.test_scenes
    datasetElements = len(test_dataset)
    for idx in scenes:
        didCollide = getCollided(agent, idx)
        if didCollide:
            subsetCollisions += 1
    totalCollisions += subsetCollisions
    print("Collision Percentage for {} agents is {}%".format(agent, subsetCollisions / datasetElements * 100.0))
    print("Total number of collisions for {} agents is {}/{}".format(agent, subsetCollisions, datasetElements))
    
print("Average Collision Percentage for all agents is {}%".format(totalCollisions / totalElements * 100.0))
print("Total number of collisions for all agents is {}/{}".format(totalCollisions, totalElements))
    